In [ ]:
import pandas as pd
import cbsodata

print(cbsodata.__version__)

table_id = '83765NED'

In [ ]:
# Downloading table list
# toc = pd.DataFrame(cbsodata.get_table_list())

tableInfos = cbsodata.get_meta(table_id=table_id, name='TableInfos')
print(table_id, ':', tableInfos[0]['Summary'])

Select all the information on my Municipality

In [82]:
filters="indexof(WijkenEnBuurten, '0809') eq 2"
dimensions_reduced = [
        'WijkenEnBuurten',
        'Codering_3',
        'AantalInwoners_5',
        'GemiddeldeWoningwaarde_35',
        'GemiddeldElektriciteitsverbruikTotaal_47',
        'GemiddeldAardgasverbruikTotaal_55',
        'BedrijfsvestigingenTotaal_78']
dimensions = [
        'WijkenEnBuurten',
        'Gemeentenaam_1',
        'SoortRegio_2', 
        'Codering_3',
        'AantalInwoners_5',
        'GemiddeldeWoningwaarde_35',
        'GemiddeldElektriciteitsverbruikTotaal_47',
        'GemiddeldAardgasverbruikTotaal_55',
        'PercentageWoningenMetStadsverwarming_63',
        'BedrijfsvestigingenTotaal_78']

wijkenEnBuurten =  pd.DataFrame(
        cbsodata.get_data(
                table_id=table_id, 
                filters=filters,
                select=dimensions_reduced))
wijkenEnBuurten

,WijkenEnBuurten,Codering_3,AantalInwoners_5,GemiddeldeWoningwaarde_35,GemiddeldElektriciteitsverbruikTotaal_47,GemiddeldAardgasverbruikTotaal_55,BedrijfsvestigingenTotaal_78
0,Loon op Zand,GM0809,23093,243.0,3420.0,1440.0,2060
1,Wijk 10 Kaatsheuvel-West,WK080910,10195,230.0,3330.0,1350.0,900
2,Draaiboom,BU08091001,2015,229.0,3490.0,1240.0,110
3,Hil (noord),BU08091005,605,220.0,3540.0,1420.0,25
4,Hil (zuid),BU08091006,740,268.0,3570.0,1360.0,45
5,Pannenhoef (west),BU08091010,1105,190.0,3010.0,1240.0,35
6,Pannenhoef (oost),BU08091011,655,192.0,2800.0,1040.0,20
7,Bloemenbuurt,BU08091015,420,180.0,3450.0,1180.0,20
8,Centrum,BU08091020,830,186.0,2650.0,1130.0,140
9,Schrijversbuurt,BU08091021,525,208.0,2840.0,1230.0,45


What looks to be a good criterium to see if we are dealing with an industrial zone, is to compare the number of companies to the number of residents.

In [87]:
wijkenEnBuurten['industial_zone'] = 2 * wijkenEnBuurten['BedrijfsvestigingenTotaal_78'] > wijkenEnBuurten['AantalInwoners_5']
wijkenEnBuurten[wijkenEnBuurten['industial_zone'] == True]

,WijkenEnBuurten,Codering_3,AantalInwoners_5,GemiddeldeWoningwaarde_35,GemiddeldElektriciteitsverbruikTotaal_47,GemiddeldAardgasverbruikTotaal_55,BedrijfsvestigingenTotaal_78,industial_zone
13,De Kets (industrieterrein),BU08091035,95,NaN,3170.0,1050.0,160,True
31,De Hoogt (Industrieterrein),BU08093085,0,NaN,NaN,NaN,40,True
34,De Plakken,BU08093092,5,NaN,NaN,NaN,5,True
